In [ ]:
import torch
from utils import get_embedding
import pandas as pd
from tqdm import tqdm
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv

In [ ]:
model_name = "text2vec-base"

In [ ]:
embedding = get_embedding(model_name=model_name)

In [ ]:
t = "刷美德：英国80后为何越来越传统"

In [ ]:
fn = "thucnews/thucnews.csv"
df = pd.read_csv(fn)
df.shape

In [ ]:
title_embeddings = torch.zeros(df.shape[0], 768)
title_embeddings.shape

In [ ]:
titles = df['title']
with torch.no_grad():
    i = 0
    for title in tqdm(titles):
        t_em = torch.tensor(embedding.embed_query(title), dtype=float)
        title_embeddings[i, :] = t_em
        i = i + 1
    print(i)

In [ ]:
with open("thucnews/views/pkl/title_embeddings.pickle", "rb") as file:
    title_embeddings = pickle.load(file)

In [ ]:
title_embeddings.shape

In [ ]:
score = cosine_similarity(title_embeddings, title_embeddings)

In [ ]:
percentage = np.sum(score > 0.4 ) / (score.size)
percentage

In [ ]:
np.sum(score >= 0.4)

In [ ]:
score[score < 0.4] = 0

In [ ]:
# ct = np.sum(score > 0, axis=1)
# for i in range(len(ct)):
#     if ct[i] == 1:
#         print("有等于零的")
#     print(i+1, ct[i])

In [ ]:
np.fill_diagonal(score, 1.0)

In [ ]:
save_path = "thucnews/views/thucnews_title.csv"
# 打开CSV文件
with open(save_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入表头
    writer.writerow(['Row', 'Column', 'Value'])

    # 遍历数组并写入CSV文件
    for i in tqdm(range(score.shape[0])):
        for j in range(score.shape[1]):
            if score[i][j] > 0:
                writer.writerow([i, j, score[i, j]])

In [ ]:
df = pd.read_csv(save_path)
df.shape

In [ ]:
df.columns

In [ ]:
df.to_csv(save_path, sep=" ", header=False, index=False)